# Using linear regression models to predict oil well locations with the highest profit margin

The mining company `OilyGiant` is looking to build `200 new oil wells`. Our task is to find the region with the highest profit margin for the company. We have information about oil reserves in three specific regions. We will train and test the model for each region in `geo_data_0.csv`.

`Conditions:`
- Only `linear regression` should be used for model training.
- When exploring the region, a `500-point study` is carried out with the `best 200 points` being selected for profit calculation.
- `The budget` for the development of `200 oil wells` is `100 million dollars`.
- `A barrel of raw materials` generates `4.5 USD of revenue`. The revenue from a `product unit is 4500 USD` (the volume of reserves is expressed in thousands of barrels).
- After risk assessment, keep only the `regions with loss risk less than 2.5%`. Of those that fit the criteria, the region with the highest average profit should be selected.
- The data is synthetic: contract details and well characteristics are not published.

Three models will be trained, one for each region (dataset). Once trained, the model will be applied to a validation dataset to forecast oil reserve levels. Based on the 200 largest oil reserves, the potential profit will be determined using the validation dataset. After the initial data has been processed and examined, an average profit distribution will be compiled for each location. For each region, the average profit, 95% confidence interval, and risk of loss will be calculated and displayed. We will choose the largest and most advantageous area for OilyGiant to build its new well based on those calculated figures.

### 1.- WE IMPORT LIBRARIES

In [267]:
# Import the necessary libraries

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import library for currency formatting
import locale

# Hide warning messages
import warnings
warnings.filterwarnings('ignore')

### 2.- LOADING THE DATA

Next, the csv files `geo_data_0.csv`, `geo_data_1.csv`, `geo_data_2.csv` will be read and stored in DataFrames.

In [268]:
# Read and store csv files into DataFrames

df_0 = pd.read_csv('../TT_Project_11/dataset/geo_data_0.csv')
df_1 = pd.read_csv('../TT_Project_11/dataset/geo_data_1.csv')
df_2 = pd.read_csv('../TT_Project_11/dataset/geo_data_2.csv')


### 3.- EXPLORING THE DATA

Let's take a look at the data stored in each of the three DataFrames. The first 15 rows of each DataFrame will be printed, followed by a summary and general information about each of them.

#### 3.1. Data Description

- `id`: unique oil well identifier
- `f0, f1, f2`: three features of the points (their specific meaning is not important, but the features themselves are significant)
- `product`: volume of reserves in the oil well (thousands of barrels).

In [269]:
datasets = [df_0, df_1, df_2]

for i in range(len(datasets)):
    print('df_' + str(i) + ':')
    print(datasets[i].head(15))
    print()

df_0:
       id        f0        f1        f2     product
0   txEyH  0.705745 -0.497823  1.221170  105.280062
1   2acmU  1.334711 -0.340164  4.365080   73.037750
2   409Wp  1.022732  0.151990  1.419926   85.265647
3   iJLyR -0.032172  0.139033  2.978566  168.620776
4   Xdl7t  1.988431  0.155413  4.751769  154.036647
5   wX4Hy  0.969570  0.489775 -0.735383   64.741541
6   tL6pL  0.645075  0.530656  1.780266   49.055285
7   BYPU6 -0.400648  0.808337 -5.624670   72.943292
8   j9Oui  0.643105 -0.551583  2.372141  113.356160
9   OLuZU  2.173381  0.563698  9.441852  127.910945
10  b8WQ6  0.371066 -0.036585  0.009208   70.326617
11  1YYm1  0.015920  1.062729 -0.722248   45.110381
12  zIYPq -0.276476  0.924865  0.095584   89.158678
13  iqTqq  0.212696 -0.111147  5.770095  164.298520
14  Ct5yY -0.018578  0.187516  2.944683  158.633720

df_1:
       id         f0         f1        f2     product
0   kBEdx -15.001348  -8.276000 -0.005876    3.179103
1   62mP7  14.272088  -3.475083  0.999183   26.

#### 3.2. Data characteristics

We use the .info() method to obtain the characteristics of the data.

In [270]:
datasets = [df_0, df_1, df_2]

for i in range(len(datasets)):
    print('df_' + str(i) + ':')
    print(datasets[i].info())
    print()

df_0:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

df_1:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

df_2:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 col

It appears that no preprocessing is required after reviewing the dataset previews and summaries. There are no missing values ​​and the data type for each column appears to be correct. We will not be modifying the data in any way as we move forward.

It should be noted that the column names are not at all descriptive. OilyGiant Mining Corporation responded that since the features had already been prepared for model training, their precise meaning was irrelevant to the task at hand.

#### 3.3 Data pre-processing:

It should be noted that each of the three datasets contains an individual identification string for each oil resource in the `id` column. Since the values ​​in the `id` column are strings, they cannot be used by a linear regression model. Therefore, it is best to remove the `id` column from each of the three datasets before you start training the models. The cell block below will remove the `id` column from all three columns.

In [271]:
df_0 = df_0.drop('id', axis=1)
df_1 = df_1.drop('id', axis=1)
df_2 = df_2.drop('id', axis=1)

## 4.- MODELING

### 4.1 Division of data sets:

Now, let us split the data, as each dataset contains only numeric values. A training dataset and a validation dataset will be created from each dataset. Each training dataset will be compared to each validation dataset in a ratio of 3:1. Therefore, the validation dataset will contain 25% of the total data, while the training dataset for each region will have 75%. The cell block below will use the `train test split` function from the `sklearn.model selection` module to split the data as instructed.

In [272]:
# Create the training, validation and testing data sets.
# The datasets will be split 75% and 25% for a 3:1 ratio between training and validation datasets.

# Initializing DataFrame features for each region
df_0_features = df_0.drop('product', axis=1)
df_1_features = df_1.drop('product', axis=1)
df_2_features = df_2.drop('product', axis=1)

# Initializing DataFrame targets for each region
df_0_target = df_0['product']
df_1_target = df_1['product']
df_2_target = df_2['product']

# Split the feature and target datasets for each DataFrame into training and validation datasets
# The test size will be 0.25 so that the validation dataset contains 25% of the data.
# and the training data set contains 75% of the data
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(df_0_features, df_0_target, test_size=0.25)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(df_1_features, df_1_target, test_size=0.25)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(df_2_features, df_2_target, test_size=0.25)

### 4.5 Training the Linear Regression Model

##### Linear Regression:
`Objective:`
Estimate a linear relationship between a dependent variable $𝑦$ and one or more independent variables $x_{1}, x_{2}, ..., x_{n}$

`Model:`
The general formula for linear regression is:


\begin{align}
y = \beta_{0}\beta_{0} + \beta_{1}x_{1} + \beta_{2}x_{2} + ... + \beta_{n}x_{n} + e
\end{align}

Where:
- $𝑦$ is the dependent variable (continuous).
- $\beta_{0}$ is the intercept.
- $\beta_{1}, \beta_{2}, ..., \beta_{n}$ are the coefficients.
- $e$ is the error term.

`Output:`
Linear regression predicts continuous values ​​for $𝑦$

Now that the training and validation datasets have been split for each regional dataset, it is finally time to train the models! Again, one model will be trained for each regional dataset. The model of choice will be a linear regression model, as we are trying to predict/calculate an accurate number in a continuous range. Since the possibilities are much more numerous than just **0** or **1**, this is not a binary classification task, so logistic regression is not an accurate model of choice for this application. The cells below will initialize a linear regression model, fit the model, predict target values ​​for the validation dataset, and then output the average volume of predicted oil reserves and the mean square error.

#### 4.5.1 Dataset 'geo_data_0':

In [273]:
# Instantiate a logistic regression model
model = LinearRegression()

# Fit the model using the training data
model.fit(features_train_0, target_train_0)

# Predict target values ​​of validation features
predicted_valid_0 = model.predict(features_valid_0)

# Calculate and print the average forecasted booking volume and model RMSE
result = mean_squared_error(target_valid_0, predicted_valid_0)**0.5
print(f"Average volume of predicted reserves: {round(predicted_valid_0.mean(),3)} thousand barrels")
print("RMSE del modelo de Regresion Lineal del conjunto de validacion_0:", round(result,3))

Volumen promedio de reservas previstas: 92.706 mil barriles
RMSE del modelo de Regresion Lineal del conjunto de validacion_0: 37.951


The average amount of expected oil reserves is 92 thousand barrels. The `root mean square error RMSE` of the linear regression model is about 37. This indicates that the standard deviation of the expected values ​​could reach 37,000 barrels.

#### 4.5.2 Dataset 'geo_data_1':

In [274]:
# Instantiate a logistic regression model
model = LinearRegression()

# Fit the model using the training data
model.fit(features_train_1, target_train_1)

# Predict target values ​​of validation features
predicted_valid_1 = model.predict(features_valid_1)

# Calculate and print the average forecasted booking volume and model RMSE
result = mean_squared_error(target_valid_1, predicted_valid_1)**0.5
print(f"Average volume of predicted reserves: {round(predicted_valid_1.mean(),3)} thousand barrels")
print("RMSE del modelo de Regresion Lineal del conjunto de validacion_1:", round(result,3))

Volumen promedio de reservas previstas: 68.462 mil barriles
RMSE del modelo de Regresion Lineal del conjunto de validacion_1: 0.89


The `average volume` of the estimated oil reserves is `68 thousand barrels`. The linear regression model has a `root mean square error RMSE` of about 0.8`. This means that the predicted values ​​can vary by as much as 0.9 thousand barrels. That RMSE value is extremely low! Because the variation in oil reserve volumes is so small, the vast majority of reserves will be close to the average of 68.

#### 4.5.3 Dataset 'geo_data_2':

In [275]:
# Instantiate a logistic regression model
model = LinearRegression()

# Fit the model using the training data
model.fit(features_train_2, target_train_2)

# Predict target values ​​of validation features
predicted_valid_2 = model.predict(features_valid_2)

# Calculate and print the average forecasted booking volume and model RMSE
result = mean_squared_error(target_valid_2, predicted_valid_2)**0.5
print(f"Average volume of predicted reserves: {round(predicted_valid_2.mean(),3)} thousand barrels")
print("RMSE del modelo de Regresion Lineal del conjunto de validacion_2:", round(result,3))

Volumen promedio de reservas previstas: 94.864 mil barriles
RMSE del modelo de Regresion Lineal del conjunto de validacion_2: 40.007


The estimated `average volume` of oil reserves is `94 thousand barrels`. The root mean square error of the linear regression model is about 40. This means that the predicted values ​​can vary by as much as 40 thousand barrels. This is a fairly large RMSE value. In fact, it has the highest RMSE of the three. The estimated volumes of oil reserves could deviate significantly from the average of 94.

#### 4.5.4 Summary

`Regions 0 and 2 have a high RSME value`, meaning that the error is high so `the accuracy is not good`, especially when `compared to region 1`. This means that the `oil reserve volumes` in regions 0 and 2 are highly variable, while those in `region 1 are more predictable`. Furthermore, the average oil reserve volume in regions 0 and 2 is approximately 1.4 times that of region 1. It will be interesting to look at the results of the profit analysis for each region, as regions 0 and 2 can generate significantly higher profits based on the distribution of their oil reserves. Region 1, on the other hand, is more likely to produce a more accurate profit range.

## 5.- CALCULATION OF BENEFITS

#### 5.1. Determine the minimum oil production required to reach the break-even point.

To begin analyzing the potential profits of each region, let’s first calculate how many thousands of barrels of oil are needed to ensure that the OilyGiant mining company is not building/operating at a loss. The cell block below will calculate the number of units, or thousands of barrels, that the oil reserve volume must provide in order for OilyGiant to make a profit. This calculation is done by dividing OilyGiant’s total budget ($100,000,000) for building 200 oil wells by the revenue generated by one unit ($4,500) of oil.

In [276]:
# Initialize constant variables
# Revenue per unit is USD 4,500 (thousand barrels)
# The budget for the development of 200 oil wells is $100,000,000.
# Revenues from 200 oil wells must exceed $100,000,000
presupuesto = 100000000 # budget 100 million
pozos_proyectados = 200
ingreso_unidad = 4500 # Reserve volume is expressed in thousands of barrels

# Volume of reserves required to develop a new well without losses:
sufficient_volume = presupuesto/ingreso_unidad

# Print the results
print(f"In total {round(sufficient_volume,3)} thousand barrels are required to avoid building/operating at a loss.")
print(f"Approximately {round((sufficient_volume / projected_wells), 3)} thousand barrels are required per reserve.")

En total se requieren 22222.222 mil barriles para evitar construir/operar sin pérdidas.
Se requieren aproximadamente 111.111 mil barriles por reserva.


Based on the above calculations, a minimum of approximately `22222.222 thousand barrels = 22.2 million barrels` of oil needs to be extracted from the new well to ensure that OilyGiant does not suffer any losses in constructing or operating the well. This translates to a minimum of about 111.11 thousand barrels from each reserve in the area. As is evident, `the projected average oil reserve volumes for the three regions, as modeled in Section 4, are less than 111.11 thousand barrels per reserve`. However, taking into account the `RMSE of Region 0 and Region 2`, it is certain that there are oil reserve volumes that `exceed the value of 111.11`. Unfortunately, `Region 1` has less oil reserves exceeding the volume of 111.11, as the `average oil reserve` volume is around `68.66 thousand barrels`, and the RMSE is about 0.887. However, if OilyGiant manages to extract oil from the top 200 oil wells in Region 1, there is a higher chance of making profits rather than incurring losses at the end of the year.

#### 5.2. Calculate potential profit from projected values

Let’s calculate the profit that OilyGiant could make based on the predicted oil reserve volumes in each region. To do this, a function will be developed that accepts the volume predictions and the sample size of oil wells in each region as arguments. The function will use the 200 largest oil wells in each region to calculate OilyGiant’s profit. First, the predictions for each region will be sorted in descending order, and then the top 200 values ​​will be extracted from the prediction series. These 200 values ​​will be added together to get a total, which will then be multiplied by $4,500 to get the total revenue. Finally, to get the final profit figure, OilyGiant’s budget will be subtracted from the product of the total reserve volume and the revenue per unit. In addition, the target number of barrels of oil that should be produced will be displayed along with the profit that OilyGiant can anticipate.

In [277]:
# Create a function to calculate profit from the predicted target values ​​for each region

# Initialization function: takes 'predictions' and 'count' as parameters
def get_profit(targets, predictions, count):
    # Sort the expected volumes from largest to smallest, number of rows `count`
    predictions_sorted = predictions.sort_values(ascending=False)[:count]
    
    # Select the 200 largest oil reserve volumes for each region, but use the target volumes (actual volumes)
    pozos_seleccionados = targets[predictions_sorted.index]
    
    # Calculate profit based on one unit producing $4,500 in revenue
    # Subtract the $100,000,000 budget from total revenue
    ganancia = (4500 * pozos_seleccionados.sum()) - presupuesto
    
    # Return the value of the profit
    return round(ganancia, 2)


# Run the following function...

# Create a list of the expected values ​​and the actual values ​​(targets) for each region
predictions = [pd.Series(predicted_valid_0), pd.Series(predicted_valid_1), pd.Series(predicted_valid_2)]
targets = [target_valid_0.reset_index(drop=True) , target_valid_1.reset_index(drop=True), target_valid_2.reset_index(drop=True)]
profits = []

# For loop to run get_profit function for the predicted data sets of each region
# Store profit values ​​in the profit list
for i in range(len(predictions)):
    profits.append(get_profit(targets[i], predictions[i], 200))

# Set local currency to USD
locale.setlocale(locale.LC_ALL, '')
    
# Print each profit value
for i in range(len(profits)):
print(f"Profit of region {i}: {locale.currency(profits[i], grouping=True)}")
print(f"Target oil reserve volume: {round((profits[i] + 100_000_000) / 4500.2)} thousand barrels\n")

Beneficio de la región 0: $36,132,948.89
Volumen objetivo de reserva de petróleo: 30251.77 mil barriles

Beneficio de la región 1: $24,150,866.97
Volumen objetivo de reserva de petróleo: 27589.08 mil barriles

Beneficio de la región 2: $26,266,615.53
Volumen objetivo de reserva de petróleo: 28059.25 mil barriles



The ranking of each region according to its estimated profits and projected oil reserve volumes is as follows:

- `1. Region 0` - `Profit`: $36 million, `Volume`: approx. 30 million barrels
- `2. Region 2` - `Profit`: $26 million, `Volume`: approx. 28 million barrels
- `3. Region 1` - `Profit`: $24 million, `Volume`: approx. 27 million barrels

As can be seen from the profits calculated above, Region 0 generated the largest profits, followed by Region 2 and then Region 1. All three regions generated profits rather than losses.

`The projected oil reserve volumes are all greater than the approximately 22.2 million barrels` required for OilyGiant to break even, so `each region is projected to generate a profit for OilyGiant`. Despite expecting Region 1 to produce fewer barrels, it appears to contain some very large oil reserves, as it produced a profit, requiring at least 200 of them. Although both Region 1 and Region 2 generated a profit for OilyGiant, `Region 0 generated the largest projected profit of over $36 million`. Therefore, based on these estimates alone, `OilyGiant should construct its new oil well in Region 0`. However, since we are relying on the larger projected reserves, which may not materialize when OilyGiant constructs its new well, it is critical to examine the distributions of average profits for each region, which will be discussed in the next section.

## 6.- APPLICATION OF BOOTSTRAPPING METHOD TO DETERMINATE THE MOST PROFITABLE REGION

While determining the profits generated by predictions of major oil reserves is a useful starting point for identifying the most profitable region, it does not provide a complete picture. By selecting the 200 largest oil reserves in each region, we obtained the maximum potential profit. To gain a more accurate understanding of the profitability of each region, let us build a distribution of average profit values ​​for each region. To do this, we will pass 1000 samples of 500 oil wells to the `get_profit` function. The function will then calculate the anticipated profit, as done above, and save the profit value to the `values` list. Once the distribution has been obtained, we will determine the average profit, 95% confidence interval, and risk of loss for each region.

In [286]:
# Use bootstrap method with 1000 samples to find the distribution of profits
# Compile an average profit distribution for each region based on the samples

def get_profit_distribution(targets, predictions, count):
    # Initialize random state
    state = np.random.RandomState(88888)
    
    # Initialize the list of values
    values = []
    
    # Create a DataFrame consisting of the prediction and target series.
    combined_df = pd.DataFrame()
    combined_df['predictions'] = predictions
    combined_df['targets'] = targets.reset_index(drop=True)
    
    # For loop to get 1000 samples and pass them to get_profit function
    for i in range(1000):
        target_subsample = combined_df.sample(n=500, replace=True, random_state=state).reset_index(drop=True)
        values.append(get_profit(target_subsample['targets'], target_subsample['predictions'], count))
    
    # Convert list of values ​​to array in pandas
    values = pd.Series(values)
    
    # Calculate the average of the earnings
    media_ganancia = values.mean()
    
    # Obtain the upper and lower limits of the 95% confidence interval
    superior = values.quantile(0.975)
    inferior = values.quantile(0.025)
    
    # Determine the probability of negative gain (loss)
    count = 0
    for value in values:
        if value < 0:
            count += 1
            
    print("Ganancia promedio:", locale.currency(media_ganancia, grouping=True))
print(f"Target oil reserve volume: {round((average_gain + 100_000_000) / 4500.2)} thousand barrels")
    print("Intervalo de Confianza del 95%:", locale.currency(inferior, grouping=True), 'to', locale.currency(superior, grouping=True))
print(f"Risk of loss: {1. * count * 100 / len(values)}%")

for i in range(len(predictions)):
    print('Region ' + str(i))
    get_profit_distribution(targets[i], predictions[i], 200)
    print()

Region 0
Ganancia promedio: $4,481,215.04
Volumen objetivo de reserva de petróleo: 23218.05 mil barriles
Intervalo de Confianza del 95%: $-935,430.13 to $9,297,603.29
Riesgo de pérdida: 4.4%

Region 1
Ganancia promedio: $4,140,035.54
Volumen objetivo de reserva de petróleo: 23142.23 mil barriles
Intervalo de Confianza del 95%: $464,983.91 to $7,901,953.60
Riesgo de pérdida: 1.5%

Region 2
Ganancia promedio: $3,451,073.19
Volumen objetivo de reserva de petróleo: 22989.13 mil barriles
Intervalo de Confianza del 95%: $-1,849,621.28 to $8,548,610.06
Riesgo de pérdida: 11.0%



`Regions 0 and 2 exceed the acceptable loss risk threshold of 2.5%,` making them unsuitable for OilyGiant's new well. Only `Region 1 has a loss risk percentage below 2.5%,` making it the only region that meets the risk criteria and can be considered for OilyGiant's new well location.

`The average profit obtained for each region through a distribution of 1000 random samples is lower than the profit estimated from the 200 largest oil reserves in each region.` However, this was to be expected. Region 1 and Region 2 have relatively high average profits, and Region 2 has a slightly lower average profit. Therefore, together with the risk of loss criterion, `OilyGiant is recommended to construct its new well in Region 1`, which has one of the highest average profits.

## 7.- CONCLUSIONS

We start by training the linear regression model with the datasets of each region to `predict the average reservation volumes` and the RMSE of each validation set of each dataset.` Obtaining that region 1 geo_data_1` has a `more accurate model but a lower average volume than region 0 and 2`.

We then `calculated the profits associated with the 200 wells` with the highest predicted values ​​and `found that region 0 (geo_data_0)` earned a `Profit` of $36 million, `Volume` of approx. 30 million barrels, with region 0`s profit being greater than regions 1 and 2, and exceeding the projected reserve volumes of 22 million barrels. While this was a good starting point for the analysis, `more samples were needed to achieve greater accuracy`.

To improve the accuracy of our models, we used the bootstrap method to obtain 1000 samples and determine the distribution of profits for each region. Using the resulting average profit, the 95% confidence interval, and the risk of loss values, we concluded that Region 1 is the most suitable location, because Region 0 and 2 exceed the risk of loss threshold of 2.5%, and the lower and upper profit confidence intervals are positive unlike the other regions.

Therefore we conclude that `region 1` is the best region for OilyGiant Mining Company to build its new oil well. This decision is made despite the fact that initially it was thought that `region 0` was the most profitable, but its model had a high RMSE, so `after applying the bootstrap method, region 1` which improves the precision of the model obtained a higher average profit and a lower risk of loss than requested.




